#Installing packages and libraries

In [ ]:
!pip install langchain openai
!pip install "weaviate-client>=3.26.7,<4.0.0"
!pip install huggingface_hub
!pip install -U datasets huggingface_hub fsspec
!pip install -U langchain-community
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.1/120.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.3/509.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 69.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.2
    Uninstalling huggingface-hub-0.31.2:
      Successfully uninstalled huggingface-hub-0.31.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uni

#Used Keys

In [ ]:
from huggingface_hub import login
login("") #insert your login key

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "" #insert your HUGGINGFACEHUB_API_TOKEN
os.environ["OPENAI_API_KEY"] = "" #insert your OPENAI_API_KEY

#Data Loading

In [ ]:
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

url = "https://huggingface.co/datasets/Kaludi/Customer-Support-Responses/blob/main/Customer-Support.csv"
res = requests.get(url)
with open("Customer-Support.csv", "w") as f:
    f.write(res.text)

loader = TextLoader('./Customer-Support.csv')
documents = loader.load()
print(documents)

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

[Document(metadata={'source': './Customer-Support.csv'}, page_content='<!doctype html>\n<html class="">\n\t<head>\n\t\t<meta charset="utf-8" />\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no" />\n\t\t<meta name="description" content="We’re on a journey to advance and democratize artificial intelligence through open source and open science." />\n\t\t<meta property="fb:app_id" content="1321688464574422" />\n\t\t<meta name="twitter:card" content="summary_large_image" />\n\t\t<meta name="twitter:site" content="@huggingface" />\n\t\t<meta name="twitter:image" content="https://cdn-thumbnails.huggingface.co/social-thumbnails/datasets/Kaludi/Customer-Support-Responses.png" />\n\t\t<meta property="og:title" content="Customer-Support.csv · Kaludi/Customer-Support-Responses at main" />\n\t\t<meta property="og:type" content="website" />\n\t\t<meta property="og:url" content="https://huggingface.co/datasets/Kaludi/Customer-Support-Responses/blob/main/Cust

In [ ]:
from datasets import load_dataset
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

# Load the dataset directly from Hugging Face Hub
dataset = load_dataset("Kaludi/Customer-Support-Responses")

# Combine Q&A into one text block
docs = [
    Document(page_content=f"Q: {row['query']}\nA: {row['response']}")
    for row in dataset["train"]
]

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Customer-Support.csv:   0%|          | 0.00/12.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/74 [00:00<?, ? examples/s]

page_content='Q: My order hasn't arrived yet.
A: We apologize for the inconvenience. Can you please provide your order number so we can investigate?'


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = embeddings,
    by_text = False
)

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.0/weaviate-v1.23.0-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 5818


<ipython-input-7-b4c1dc005abf>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Retrieving

Now that we have our external dataset, we are going to use it as the retriever for the LLM to fetch relevant data

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 1, "lambda_mult": 0.5})
#retriever = vectorstore.as_retriever()

#Augment

We are now going to give the LLM a template prompt for context, to increase it's response efficiency

In [ ]:
from langchain.prompts import ChatPromptTemplate

template_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template_prompt)

#Text Generation

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.output_parser import StrOutputParser
from langchain import HuggingFaceHub
from langchain.llms import OpenAI
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-base", max_length=500)
llm = HuggingFacePipeline(pipeline=pipe)

#check which documents (QAs) are retrieved
docs = retriever.invoke("How do I reset my password?")
for i, doc in enumerate(docs):
    print(f"\n--- Document {i+1} ---\n{doc.page_content}")

def format_context(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever | RunnableLambda(format_context), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Device set to use cpu



--- Document 1 ---
Q: How do I reset my password?
A: We can help with that. Please provide your account email, and we'll send you instructions on how to reset your password.

--- Formatted Context ---
 Q: How do I reset my password?
A: We can help with that. Please provide your account email, and we'll send you instructions on how to reset your password.


In [ ]:
query = "How do I reset my password?"
print(rag_chain.invoke(query))

Please provide your account email, and we'll send you instructions on how to reset your password.


#Deploy Locally

##Required Libraries

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 65.3 MB/s eta 0:00:00


We will use gradio to deploy as we are only demonstrating how it would work

In [ ]:
import gradio as gr

def answer_question(q):
    return rag_chain.invoke(q)

gr.Interface(fn=answer_question, inputs="text", outputs="text").launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0c310283cb7f070844.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
